## google email 


jeans site:amazon.com "@gmail.com"


普遍邮箱适用的正则表达式

reg = r'[A-Za-z0-9_]+@[A-Za-z0-9]+.[a-z]{3}

因为

\w 等同于[a-z0-9A-Z_]匹配大小写字母、数字和下划线

所以

reg = r'\w+@\w+.[a-z]{3}

In [ ]:
import time,random,re,csv
from pyquery import PyQuery as pq


def email(url):

    doc = pq(url)
    #print(doc)
    content = doc('.st')
    print(content)
    

url = 'https://www.google.com.hk/search?safe=strict&biw=1920&bih=889&ei=d3NKXMXkF6LB7gLp6ZX4Dw&q=jeans+site%3Aamazon.com+%22%40gmail.com%22&oq=jeans+site%3Aamazon.com+%22%40gmail.com%22&gs_l=psy-ab.3...9441.11399..11908...0.0..0.237.2275.2-10......0....1..gws-wiz.AJwyto52dO4'
email(url)

### 省略

为了向您显示相关程度最高的结果，我们省略了一些与已显示的 14 条结果极为相似的条目。
如有需要，您可以重新搜索以显示省略的结果。

In [18]:
from selenium import webdriver
import time,random,re,csv,pymongo
from pyquery import PyQuery as pq
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

    
def get_email(driver,url,save_file,page_num):
    
    driver.get(url) # 访问网页
    print('open crawl page... {}'.format(url))
    time.sleep(2)    
    jsCode = "var q=document.documentElement.scrollTop=100000"
    driver.execute_script(jsCode)
    html = driver.page_source
    doc = pq(html)
    current_page = int(driver.find_element_by_css_selector('td.cur').text)
    print('current page is {}'.format(current_page))
    
    content = doc.find('.st')
    with open(save_file,'a',encoding='utf-8') as w:
        for i in content.items():
            text = re.sub(r'\n','',pq(i.html()).text()) 
            try:
                email = re.search(r'\w+@\w+.[a-z]{3}',text).group() 
                print('email found {}'.format(email))
                search = {
                    'email': email
                }
                save_to_mongo(search)
                #w.write('{}\n'.format(email))
            except AttributeError:
                print('no email found')
    
    try:
        next_page = driver.find_element_by_id('pnnext').get_attribute('href')
        if current_page < page_num :
            print('next page url: {}'.format(next_page))
            time.sleep(random.randint(2,4))
            get_email(driver,next_page,save_file,page_num)     
    except NoSuchElementException:
        print('there is no more pages.')
        
    
    try:
        next_page = driver.find_element_by_id('pnnext').get_attribute('href')
        if current_page < page_num :
            print('next page url: {}'.format(next_page))
            time.sleep(random.randint(2,4))
            get_email(driver,next_page,save_file,page_num)     
    except NoSuchElementException:
        print('there is no more pages.')    
    
            
def save_to_mongo(result):
    MONGO_URL = 'localhost'
    MONGO_DB = 'shopcrawl'
    MONGO_COLLECTION = 'email'
    client = pymongo.MongoClient(MONGO_URL)
    db = client[MONGO_DB]
    collection = db[MONGO_COLLECTION]
    try:
        if collection.insert_one(result):
            print('save to mongodb')
    except Exception:
        print('failed to save to mongodb')


#url = 'https://www.google.com.hk/search?safe=strict&biw=1920&bih=889&ei=d3NKXMXkF6LB7gLp6ZX4Dw&q=jeans+site%3Aamazon.com+%22%40gmail.com%22&oq=jeans+site%3Aamazon.com+%22%40gmail.com%22&gs_l=psy-ab.3...9441.11399..11908...0.0..0.237.2275.2-10......0....1..gws-wiz.AJwyto52dO4'
url_1 = 'https://www.google.com.hk/search?safe=strict&ei=KYpKXKzPCPXY9AOd44GwBw&q=jeans+site%3Awish.com+%22%40gmail.com%22&oq=jeans+site%3Awish.com+%22%40gmail.com%22&gs_l=psy-ab.3...8574389.8575320..8575770...0.0..0.225.1094.2-5......0....1..gws-wiz.TUS1hiC8cxo'
save_file = r'email_result.csv'
page_num = 50

def goole_email(url,save_file,page_num):
    
    #chrome_options = Options()
    #chrome_options.add_argument('--headless')
    #driver = webdriver.Chrome(chrome_options=chrome_options) 
    driver = webdriver.Chrome() 
    get_email(driver,url,save_file,page_num)
    driver.close()
    print('getting email done')
    
goole_email(url_1,save_file,page_num)

open crawl page... https://www.google.com.hk/search?safe=strict&ei=KYpKXKzPCPXY9AOd44GwBw&q=jeans+site%3Awish.com+%22%40gmail.com%22&oq=jeans+site%3Awish.com+%22%40gmail.com%22&gs_l=psy-ab.3...8574389.8575320..8575770...0.0..0.225.1094.2-5......0....1..gws-wiz.TUS1hiC8cxo
current page is 1
email found budbittner16@gmail.com
save to mongodb
no email found
no email found
email found miaantunez1987@gmail.com
save to mongodb
email found miaantunez1987@gmail.com
save to mongodb
email found justaregularemail@gmail.com
save to mongodb
email found hrosser90@gmail.com
save to mongodb
no email found
no email found
next page url: https://www.google.com.hk/search?q=jeans+site:wish.com+%22%40gmail.com%22&safe=strict&ei=QcdKXNSVL5O50PEPmsODYA&start=10&sa=N&ved=0ahUKEwiUuq23wIjgAhWTHDQIHZrhAAwQ8NMDCH8
open crawl page... https://www.google.com.hk/search?q=jeans+site:wish.com+%22%40gmail.com%22&safe=strict&ei=QcdKXNSVL5O50PEPmsODYA&start=10&sa=N&ved=0ahUKEwiUuq23wIjgAhWTHDQIHZrhAAwQ8NMDCH8
current page

In [ ]:
from selenium import webdriver
import time,random,re,csv,pymongo
from pyquery import PyQuery as pq
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

    
def get_email(driver,url,save_file,page_num):
    
    driver.get(url) # 访问网页
    print('open crawl page... {}'.format(url))
    time.sleep(2)    
    jsCode = "var q=document.documentElement.scrollTop=100000"
    driver.execute_script(jsCode)
    html = driver.page_source
    doc = pq(html)
    current_page = int(driver.find_element_by_css_selector('td.cur').text)
    print('current page is {}'.format(current_page))
    
    content = doc.find('.st')
    with open(save_file,'a',encoding='utf-8') as w:
        for i in content.items():
            text = re.sub(r'\n','',pq(i.html()).text()) 
            try:
                email = re.search(r'\w+@\w+.[a-z]{3}',text).group() 
                print('email found {}'.format(email))
                search = {
                    'email': email
                }
                save_to_mongo(search)
                #w.write('{}\n'.format(email))
            except AttributeError:
                print('no email found')
    
    try:
        next_page = driver.find_element_by_id('ofr i a').get_attribute('href')
        if current_page < page_num :
            print('next page url: {}'.format(next_page))
            time.sleep(random.randint(2,4))
            get_email(driver,next_page,save_file,page_num)     
    except NoSuchElementException:
        print('there is no more pages.')
        
    
    
    try:
        next_page = driver.find_element_by_id('pnnext').get_attribute('href')
        if current_page < page_num :
            print('next page url: {}'.format(next_page))
            time.sleep(random.randint(2,4))
            get_email(driver,next_page,save_file,page_num)     
    except NoSuchElementException:
        print('there is no more pages.')    
    
            
def save_to_mongo(result):
    MONGO_URL = 'localhost'
    MONGO_DB = 'shopcrawl'
    MONGO_COLLECTION = 'email'
    client = pymongo.MongoClient(MONGO_URL)
    db = client[MONGO_DB]
    collection = db[MONGO_COLLECTION]
    try:
        if collection.insert_one(result):
            print('save to mongodb')
    except Exception:
        print('failed to save to mongodb')


#url = 'https://www.google.com.hk/search?safe=strict&biw=1920&bih=889&ei=d3NKXMXkF6LB7gLp6ZX4Dw&q=jeans+site%3Aamazon.com+%22%40gmail.com%22&oq=jeans+site%3Aamazon.com+%22%40gmail.com%22&gs_l=psy-ab.3...9441.11399..11908...0.0..0.237.2275.2-10......0....1..gws-wiz.AJwyto52dO4'
url_1 = 'https://www.google.com.hk/search?safe=strict&ei=KYpKXKzPCPXY9AOd44GwBw&q=jeans+site%3Awish.com+%22%40gmail.com%22&oq=jeans+site%3Awish.com+%22%40gmail.com%22&gs_l=psy-ab.3...8574389.8575320..8575770...0.0..0.225.1094.2-5......0....1..gws-wiz.TUS1hiC8cxo'
save_file = r'email_result.csv'
page_num = 50

def goole_email(url,save_file,page_num):
    
    #chrome_options = Options()
    #chrome_options.add_argument('--headless')
    #driver = webdriver.Chrome(chrome_options=chrome_options) 
    driver = webdriver.Chrome() 
    get_email(driver,url,save_file,page_num)
    driver.close()
    print('getting email done')
    
goole_email(url_1,save_file,page_num)

In [19]:
from selenium import webdriver
import time,random,re,csv,pymongo
from pyquery import PyQuery as pq
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

    
def get_email(driver,url,save_file,page_num):
    
    driver.get(url) # 访问网页
    print('open crawl page... {}'.format(url))
    time.sleep(2)    
    jsCode = "var q=document.documentElement.scrollTop=100000"
    driver.execute_script(jsCode)
    html = driver.page_source
    doc = pq(html)
    current_page = int(driver.find_element_by_css_selector('td.cur').text)
    print('current page is {}'.format(current_page))
    
    content = doc.find('.st')
    with open(save_file,'a',encoding='utf-8') as w:
        for i in content.items():
            text = re.sub(r'\n','',pq(i.html()).text()) 
            try:
                email = re.search(r'\w+@\w+.[a-z]{3}',text).group() 
                print('email found {}'.format(email))
                search = {
                    'email': email
                }
                save_to_mongo(search)
                #w.write('{}\n'.format(email))
            except AttributeError:
                print('no email found')
    
    try:
        next_page = driver.find_element_by_id('pnnext').get_attribute('href')
        if current_page < page_num :
            print('next page url: {}'.format(next_page))
            time.sleep(random.randint(2,4))
            get_email(driver,next_page,save_file,page_num)     
    except NoSuchElementException:
        print('there is no more pages.')

            
def save_to_mongo(result):
    MONGO_URL = 'localhost'
    MONGO_DB = 'shopcrawl'
    MONGO_COLLECTION = 'email'
    client = pymongo.MongoClient(MONGO_URL)
    db = client[MONGO_DB]
    collection = db[MONGO_COLLECTION]
    try:
        if collection.insert_one(result):
            print('save to mongodb')
    except Exception:
        print('failed to save to mongodb')


url = 'https://www.google.com.hk/search?safe=strict&biw=1920&bih=889&ei=d3NKXMXkF6LB7gLp6ZX4Dw&q=jeans+site%3Aamazon.com+%22%40gmail.com%22&oq=jeans+site%3Aamazon.com+%22%40gmail.com%22&gs_l=psy-ab.3...9441.11399..11908...0.0..0.237.2275.2-10......0....1..gws-wiz.AJwyto52dO4'
save_file = r'email_result.csv'
page_num = 50

def goole_email(url,save_file,page_num):
    
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    driver = webdriver.Chrome(chrome_options=chrome_options) 
    #driver = webdriver.Chrome() 
    get_email(driver,url,save_file,page_num)
    driver.close()
    print('getting email done')
    
goole_email(url,save_file,page_num)

open crawl page... https://www.google.com.hk/search?safe=strict&biw=1920&bih=889&ei=d3NKXMXkF6LB7gLp6ZX4Dw&q=jeans+site%3Aamazon.com+%22%40gmail.com%22&oq=jeans+site%3Aamazon.com+%22%40gmail.com%22&gs_l=psy-ab.3...9441.11399..11908...0.0..0.237.2275.2-10......0....1..gws-wiz.AJwyto52dO4
current page is 1
email found lbts@gmail.com
save to mongodb
email found magekagura9@gmail.com
save to mongodb
email found saadbenchkr@gmail.com
save to mongodb
email found dakilivewallpapers@gmail.com
save to mongodb
email found breakthewineglass@gmail.com
save to mongodb
email found mikimarcu256@gmail.com
save to mongodb
email found Jsokolis@gmail.com
save to mongodb
email found breakthewineglass@gmail.com
save to mongodb
email found evertvear@gmail.com
save to mongodb
next page url: https://www.google.com.hk/search?q=jeans+site:amazon.com+%22%40gmail.com%22&safe=strict&biw=1920&bih=889&ei=18pKXLLdBsXo9AOb7rmoBQ&start=10&sa=N&ved=0ahUKEwjypOPsw4jgAhVFNH0KHRt3DlUQ8NMDCHc
open crawl page... https://www.

current page is 10
email found msgajraj1984@gmail.com
save to mongodb
email found eddieriva@gmail.com
save to mongodb
email found msf0wl3@gmail.com
save to mongodb
email found visanuw86@gmail.com
save to mongodb
email found msgajraj1984@gmail.com
save to mongodb
email found madhukar262@gmail.com
save to mongodb
email found ahailu10@gmail.com
save to mongodb
email found aruboy56@gmail.com
save to mongodb
email found delisafashion2020@gmail.com
save to mongodb
email found Fashionscrubs4u@gmail.com
save to mongodb
next page url: https://www.google.com.hk/search?q=jeans+site:amazon.com+%22%40gmail.com%22&safe=strict&biw=1920&bih=889&ei=FstKXMjjHPjC0PEP--uSsAc&start=100&sa=N&ved=0ahUKEwjIxv6KxIjgAhV4ITQIHfu1BHY4WhDw0wMIaA
open crawl page... https://www.google.com.hk/search?q=jeans+site:amazon.com+%22%40gmail.com%22&safe=strict&biw=1920&bih=889&ei=FstKXMjjHPjC0PEP--uSsAc&start=100&sa=N&ved=0ahUKEwjIxv6KxIjgAhV4ITQIHfu1BHY4WhDw0wMIaA
current page is 11
email found amorebeaute2012@gmail.com
sa

open crawl page... https://www.google.com.hk/search?q=jeans+site:amazon.com+%22%40gmail.com%22&safe=strict&biw=1920&bih=889&ei=UMtKXObwCoCR0PEPnNe36AQ&start=190&sa=N&ved=0ahUKEwjm2MCmxIjgAhWACDQIHZzrDU04tAEQ8NMDCGQ
current page is 20
email found user@gmail.com
save to mongodb
email found thatsgrandluxuries@gmail.com
save to mongodb
email found ReviewOne23@gmail.com
save to mongodb
email found vinodsb25@gmail.com
save to mongodb
no email found
email found harlesshouse3@gmail.com
save to mongodb
email found cohenmayer7@gmail.com
save to mongodb
email found bengoosales@gmail.com
save to mongodb
email found delisafashion2020@gmail.com
save to mongodb
email found barryzrose@gmail.com
save to mongodb
next page url: https://www.google.com.hk/search?q=jeans+site:amazon.com+%22%40gmail.com%22&safe=strict&biw=1920&bih=889&ei=V8tKXIq9EK3M0PEP3Iu2wAE&start=200&sa=N&ved=0ahUKEwjKxPGpxIjgAhUtJjQIHdyFDRg4vgEQ8NMDCG0
open crawl page... https://www.google.com.hk/search?q=jeans+site:amazon.com+%22%40gma